In [ ]:
import pandas as pd

# read txt
with open('castToTxt.txt', 'r') as file:
    lines = file.readlines()

# using a DF
data = []


# skip head line
for line in lines[1:]:
    parts = line.split(maxsplit=2)  # split into 3 parts: Ser, Category, SubCategory
    if len(parts) == 2: # SubCategory missing
        ser, category = parts
        sub_category = None
    elif len(parts) == 3:
        ser, category, sub_category = parts
    data.append([int(ser), category, sub_category])

# add tp DF
df = pd.DataFrame(data, columns=['Ser', 'Category', 'SubCategory'])

# print df
print(df)